In [3]:
import pandas
import bs4
import requests
import warnings
import pandas as pd

# Ignore all warnings
warnings.filterwarnings("ignore")


#### t20_wc_match_results

In [111]:

url="https://www.espncricinfo.com/records/tournament/team-match-results/icc-men-s-t20-world-cup-2022-23-14450"
res=requests.get(url,verify=False)
text=res.content
masterlink=[]
diclink={}
soup=bs4.BeautifulSoup(text)
matches=[]
table=soup.find_all('table',attrs={'class':'ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide'})
all_rows=table[0].find_all('tr')
for tr in all_rows[1:]:
    all_tds=tr.find_all('td')
    for span in all_tds:
        all_spans=span.find_all('span')
        content=all_spans[0].text
        a=span.find_all("a",attrs={'class':'ds-inline-flex ds-items-start ds-leading-none'})
        if a!=[]:
            a_tag = span.find('a')
            link = a_tag.get('href')
            if link.startswith('/series'):
                masterlink.append(link)
        # print(content)
        if content.startswith('T20I'):
            # print("hello")
            diclink[link]=content
        matches.append(content)


In [112]:
# Initialize empty lists to store extracted data
team_1 = []
team_2 = []
winner = []
margin = []
ground = []
match_date = []
scorecard = []

# Loop through the matches list in steps of 7
for i in range(0, len(matches), 7):
    team_1.append(matches[i])        # Team 1
    team_2.append(matches[i+1])      # Team 2
    winner.append(matches[i+2])      # Winner
    margin.append(matches[i+3])      # Margin
    ground.append(matches[i+4])      # Ground
    match_date.append(matches[i+5])  # Match Date
    scorecard.append(matches[i+6])   # Scorecard

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Team 1': team_1,
    'Team 2': team_2,
    'Winner': winner,
    'Margin': margin,
    'Ground': ground,
    'Match Date': match_date,
    'Scorecard': scorecard
})
df=df.rename(columns={'Team 1':'team1','Team 2':'team2','Winner':'winner','Margin':'margin','Ground':'ground',"Match Date":"matchDate","Scorecard":"match_id"})
# Save the DataFrame to a CSV file
df.to_csv('dim_match_summary.csv', index=False)

#### fact_batting_summary

In [113]:
df_concat_batting=pd.DataFrame()
for url1 in masterlink:
    url="https://www.espncricinfo.com"+url1
    match_id1=diclink[url1]
    res=requests.get(url,verify=False)
    text=res.content
    soup=bs4.BeautifulSoup(text)
    match=soup.find_all('h1')
    match=match[0].text.split(",")[0]
    a=soup.find('span',attrs={'class':'ds-text-title-xs ds-font-bold ds-capitalize'})
    innings=a.text
    batting=[]
    stats=[]
    table=soup.find_all('table',attrs={"class":"ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table"})
    all_rows=table[0].find_all('tr')
    for tr in all_rows:
        all_tds=tr.find_all('td')
        for div in all_tds:
            batsman=div.text
            batsman = batsman.replace('\xa0', '').replace('â€ ','')
            batting.append(batsman)
        batting.append(match_id1)
        batting.append(match)
        batting.append(innings)
        
    batting = [item for item in batting if item != '']

    if 'Extras' in batting:
        index = batting.index('Extras')
        lst = batting[:index] 
    
    # Initialize an empty result list
    result = []

    i = 0
    while i < len(lst):
        # Check if the next two elements are 'Pakistan vs England', 'England', followed by another pair
        if i < len(lst) - 4 and lst[i+1] == match and lst[i + 2] == innings and lst[i + 4] == match and lst[i + 5] == innings and lst[i]==match_id1 and lst[i+3]==match_id1:
            result.append(match_id1)
            result.append(match)
            result.append(innings)
            i += 6  # Skip the next duplicate pair
        else:
            result.append(lst[i])
            i += 1
    matches=result[:-3].copy()
    
    # Initialize empty lists to store extracted data
    match=[]
    batter = []
    out_by = []
    r = []
    b = []
    m = []
    fours = []
    sixes = []
    strikerate=[]
    innings=[]
    matchid=[]
    for i in range(0, len(matches), 11):
        matchid.append(matches[i])
        match.append(matches[i+1]) 
        innings.append(matches[i+2]) 
        batter.append(matches[i+3])       
        out_by.append(matches[i+4])     
        r.append(matches[i+5])    
        b.append(matches[i+6])     
        m.append(matches[i+7])    
        fours.append(matches[i+8])  
        sixes.append(matches[i+9])   
        strikerate.append(matches[i+10]) 
        
    # Create a DataFrame from the lists
    df1 = pd.DataFrame({
        'match':match,
        'teamInnings':innings,
        'match_id':matchid,
        'batsmanName': batter,
        'Out By': out_by,
        'runs': r,
        'balls': b,
        'm': m,
        '4s': fours,
        '6s': sixes,
        'SR':strikerate})
    df1=df1.drop(['Out By','m'],axis=1)
    df1['battingPos'] = df1.index + 1
    df_concat_batting=pd.concat([df1,df_concat_batting], ignore_index=True)
    # print(df_concat_batting.head())

In [114]:
for url1 in masterlink:
    url="https://www.espncricinfo.com"+url1
    match_id1=diclink[url1]
    res=requests.get(url,verify=False)
    text=res.content
    soup=bs4.BeautifulSoup(text)
    match=soup.find_all('h1')
    match=match[0].text.split(",")[0]
    a=soup.find_all('span',attrs={'class':'ds-text-title-xs ds-font-bold ds-capitalize'})
    innings=a[1].text
    batting=[]
    stats=[]
    table=soup.find_all('table',attrs={"class":"ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table"})
    all_rows=table[1].find_all('tr')
    for tr in all_rows:
        all_tds=tr.find_all('td')
        for div in all_tds:
            batsman=div.text
            batsman = batsman.replace('\xa0', '').replace('â€','')
            batting.append(batsman)
        batting.append(match_id1)
        batting.append(match)
        batting.append(innings)
    batting = [item for item in batting if item != '']

    if 'Extras' in batting:
        index = batting.index('Extras')
        lst = batting[:index] 
    
    # Initialize an empty result list
    result = []

    i = 0
    while i < len(lst):
        # Check if the next two elements are 'Pakistan vs England', 'England', followed by another pair
        if i < len(lst) - 4 and lst[i+1] == match and lst[i + 2] == innings and lst[i + 4] == match and lst[i + 5] == innings and lst[i]==match_id1 and lst[i+3]==match_id1:
            result.append(match_id1)
            result.append(match)
            result.append(innings)
            i += 6  # Skip the next duplicate pair
        else:
            result.append(lst[i])
            i += 1
    matches=result[:-3].copy()
    
    # Initialize empty lists to store extracted data
    match=[]
    batter = []
    out_by = []
    r = []
    b = []
    m = []
    fours = []
    sixes = []
    strikerate=[]
    innings=[]
    matchid=[]
    for i in range(0, len(matches), 11):
        matchid.append(matches[i])
        match.append(matches[i+1]) 
        innings.append(matches[i+2]) 
        batter.append(matches[i+3])       
        out_by.append(matches[i+4])     
        r.append(matches[i+5])    
        b.append(matches[i+6])     
        m.append(matches[i+7])    
        fours.append(matches[i+8])  
        sixes.append(matches[i+9])   
        strikerate.append(matches[i+10]) 
        
    # Create a DataFrame from the lists
    df2 = pd.DataFrame({
        'match':match,
        'teamInnings':innings,
        'match_id':matchid,
        'batsmanName': batter,
        'Out By': out_by,
        'runs': r,
        'balls': b,
        'm': m,
        '4s': fours,
        '6s': sixes,
        'SR':strikerate})
    df2=df2.drop(['Out By','m'],axis=1)
    df2['battingPos'] = df2.index + 1
    df_concat_batting=pd.concat([df2,df_concat_batting], ignore_index=True)
    

In [115]:
df_concat_batting= df_concat_batting.sort_values(by=['match','teamInnings'])
df_concat_batting.to_csv('fact_batting_summary.csv', index=False)
df=pd.read_csv("fact_batting_summary.csv")

### fact_bowling_summary

In [116]:
df_concat_bowling=pd.DataFrame()
for url1 in masterlink:
    url="https://www.espncricinfo.com"+url1
    match_id1=diclink[url1]
    res=requests.get(url,verify=False)
    content=[]
    text=res.content
    soup=bs4.BeautifulSoup(text)
    match=soup.find_all('h1')
    match=match[0].text.split(",")[0]
    team1=soup.find_all('span',attrs={'class':'ds-text-title-xs ds-font-bold ds-capitalize'})
    battingteam=team1[0].text
    bowlingteam=team1[1].text

    teamm=battingteam + " vs " + bowlingteam
    table=soup.find('table',attrs={'class':'ds-w-full ds-table ds-table-md ds-table-auto'})
    all_rows=table.find_all('tr')

    for tr in all_rows:
        all_tds=tr.find_all('td')
        content.append(match_id1)
        content.append(match)
        content.append(bowlingteam)
        for tds in all_tds:
            content.append(tds.text)
            
    
    lst= [item for item in content if len(item) <= 30]
    result=[]
    i = 0
    if lst[-1]==bowlingteam:
        lst=lst[:-3]
    while i < len(lst):
        if i < len(lst) - 4 and lst[i+1] == match and lst[i + 2] == bowlingteam and lst[i + 4] == match and lst[i + 5] == bowlingteam and lst[i]==match_id1 and lst[i+3]==match_id1:
            result.append(match_id1)
            result.append(match)
            result.append(bowlingteam)
            i += 6  # Skip the next duplicate pair
        else:
            result.append(lst[i])
            i += 1
 
    matches=result
    
    # Initialize empty lists to store extracted data
    matchid=[]
    match=[]	
    bowlingTeam=[]	
    bowlerName=[]	
    overs=[]	
    maiden=[]	
    runs=[]	
    wickets=[]	
    economy=[]	
    zeros=[]	
    fours=[]	
    sixes=[]	
    wides=[]	
    noBalls=[]	

    for i in range(0, len(matches), 14):
        matchid.append(matches[i]) 
        match.append(matches[i+1])       
        bowlingTeam.append(matches[i+2])     
        bowlerName.append(matches[i+3])    
        overs.append(matches[i+4])     
        maiden.append(matches[i+5])    
        runs.append(matches[i+6])  
        wickets.append(matches[i+7])   
        economy.append(matches[i+8])  
        zeros.append(matches[i+9]) 
        fours.append(matches[i+10]) 
        sixes.append(matches[i+11]) 
        wides.append(matches[i+12]) 
        noBalls.append(matches[i+13])  

    # Create a DataFrame from the lists
    df1 = pd.DataFrame({
        'match_id':matchid,
        'match': match,
        'bowlingTeam': bowlingTeam,
        'bowlerName': bowlerName,
        'overs': overs,
        'maiden': maiden,
        'runs' : runs,
        'wickets': wickets,
        'economy': economy,
        '0s':zeros,
        '4s':fours,
        '6s':sixes,
        'wides':wides,
        'noBalls':noBalls
    })
    
    df_concat_bowling=pd.concat([df1,df_concat_bowling], ignore_index=True)
    

In [117]:
for url1 in masterlink:
    url="https://www.espncricinfo.com"+url1
    match_id1=diclink[url1]
    
    res=requests.get(url,verify=False)
    content=[]
    text=res.content
    soup=bs4.BeautifulSoup(text)
    match=soup.find_all('h1')
    match=match[0].text.split(",")[0]

    team1=soup.find_all('span',attrs={'class':'ds-text-title-xs ds-font-bold ds-capitalize'})
    battingteam=team1[1].text
    bowlingteam=team1[0].text
    teamm= bowlingteam +" vs " + battingteam 

    content.append(match)
    content.append(bowlingteam)
    table=soup.find_all('table',attrs={'class':'ds-w-full ds-table ds-table-md ds-table-auto'})
    all_rows=table[1].find_all('tr')

    for tr in all_rows:
        all_tds=tr.find_all('td')
        for tds in all_tds:
            content.append(tds.text)
        content.append(match_id1)
        content.append(match)
        content.append(bowlingteam)
    
    lst= [item for item in content if len(item) <= 30]
    result=[]
    i = 0
    if lst[-1]==bowlingteam:
        lst=lst[:-3]
    while i < len(lst):
        if i < len(lst) - 4 and lst[i+1] == match and lst[i + 2] == bowlingteam and lst[i + 4] == match and lst[i + 5] == bowlingteam and lst[i]==match_id1 and lst[i+3]==match_id1:
            result.append(match_id1)
            result.append(match)
            result.append(bowlingteam)
            i += 6  # Skip the next duplicate pair
        else:
            result.append(lst[i])
            i += 1
            
     # Initialize empty lists to store extracted data
    matchid=[]
    match=[]	
    bowlingTeam=[]	
    bowlerName=[]	
    overs=[]	
    maiden=[]	
    runs=[]	
    wickets=[]	
    economy=[]	
    zeros=[]	
    fours=[]	
    sixes=[]	
    wides=[]	
    noBalls=[]	

    for i in range(0, len(matches), 14):
        matchid.append(matches[i]) 
        match.append(matches[i+1])       
        bowlingTeam.append(matches[i+2])     
        bowlerName.append(matches[i+3])    
        overs.append(matches[i+4])     
        maiden.append(matches[i+5])    
        runs.append(matches[i+6])  
        wickets.append(matches[i+7])   
        economy.append(matches[i+8])  
        zeros.append(matches[i+9]) 
        fours.append(matches[i+10]) 
        sixes.append(matches[i+11]) 
        wides.append(matches[i+12]) 
        noBalls.append(matches[i+13])  

    # Create a DataFrame from the lists
    df2 = pd.DataFrame({
        'match_id':matchid,
        'match': match,
        'bowlingTeam': bowlingTeam,
        'bowlerName': bowlerName,
        'overs': overs,
        'maiden': maiden,
        'runs' : runs,
        'wickets': wickets,
        'economy': economy,
        '0s':zeros,
        '4s':fours,
        '6s':sixes,
        'wides':wides,
        'noBalls':noBalls
    })
    
    df_concat_bowling=pd.concat([df2,df_concat_bowling], ignore_index=True)
    

In [118]:
df_concat_bowling= df_concat_bowling.sort_values(by=['match','bowlingTeam'])
df_concat_bowling.to_csv('fact_bowling_summary.csv', index=False)
df=pd.read_csv("fact_bowling_summary.csv")

### dim players

In [6]:
url="https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/squads"
res=requests.get(url,verify=False)
text=res.content
soup=bs4.BeautifulSoup(text)
matches=[]
hrefs = [a['href'] for a in soup.find_all('a', href=True)]
hrefs = [element for element in hrefs if element.endswith("/series-squads")]
links=[]
start="https://www.espncricinfo.com"
for i in hrefs:
    links.append(start+i)

In [7]:
players=[]
for i in links:
    url=i
    res=requests.get(url,verify=False)
    text=res.content
    soup=bs4.BeautifulSoup(text)
    matches=[]
    p=soup.find('div',attrs={'class':'ds-w-full ds-bg-fill-content-prime ds-overflow-hidden ds-rounded-xl ds-border ds-border-line'})
    hrefs = list(set([a['href'] for a in p.find_all('a', href=True)]))
    hrefs = [element for element in hrefs if element.startswith("/cricketers/")]
    for i in hrefs:
        players.append(start+i)

In [11]:
df_players=pd.DataFrame()
for url in players[:1]:
    res=requests.get(url,verify=False)
    text=res.content
    soup=bs4.BeautifulSoup(text)
    matches=[]
    header=soup.find_all('p',attrs={'class':'ds-text-tight-m ds-font-regular ds-uppercase ds-text-typo-mid3'})
    value=soup.find_all('span',attrs={'class':'ds-text-title-s ds-font-bold ds-text-typo'})
    dic={}
    for i in range(0,len(header)):
        dic[header[i].text]=value[i].text
   
    name = dic.get('Full Name', '')
    battingStyle = dic.get('Batting Style', '')
    bowlingStyle = dic.get('Bowling Style', '')
    playingRole = dic.get('Playing Role', '')
    team=soup.find('span',attrs={'class':'ds-text-title-s ds-font-bold ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block'}).text
    image_url = soup.find_all('img')
    print(image_url)
    # Create a DataFrame from the lists
    data={
        'name': name,
        'team': team,
        'battingStyle': battingStyle,
        'bowlingStyle': bowlingStyle,
        'playingRole': playingRole,
        'image': image_url,
        'description':""
    }

    df = pd.DataFrame([data])
    df_players=pd.concat([df_players,df], ignore_index=True)


[<img alt="CAN Flag" class="ds-mr-2" height="20" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="width:20px;height:20px" width="20"/>, <img alt="NEP Flag" class="ds-mr-2" height="20" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="width:20px;height:20px" width="20"/>, <img alt="HANTS Flag" class="ds-mr-2" height="20" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="width:20px;height:20px" width="20"/>, <img alt="WORCS Flag" class="ds-mr-2" height="20" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="width:20px;height:20px" width="20"/>, <img alt="NOTTS Flag" class="ds-mr-2" height="20" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="width:20px;height:20px" width="20"/>, <img alt="KENT Flag" class="ds-mr-2" height="20" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="width:20px;height:20px" widt

In [122]:
df_players.to_csv("dim_players.csv",index=False)
df=pd.read_csv("dim_players.csv")